In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 해당 웹 페이지 열기
driver.get("https://heroesbaseball.co.kr/games/schedule/list1st.do")

year_select = Select(driver.find_element(By.ID, 'searchYear'))
year_select.select_by_value('2023')  # 2024년 선택

# 월 드롭다운 클릭 (드롭다운이 바로 클릭되도록 처리)
month_select = driver.find_element(By.ID, 'searchMonth')
month_select.select_by_value('7')

# "이동" 버튼 클릭
move_button = driver.find_element(By.CLASS_NAME, 'submit')
move_button.click()

# 페이지 로딩 대기 (경기 정보가 로드될 때까지 대기)
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".gameList"))
)

# 경기 일정 테이블 가져오기
time.sleep(2)  # 일정이 로드될 때까지 대기

# 각 경기 정보를 추출
game_cells = driver.find_elements(By.CSS_SELECTOR, ".gameList td")

# 각 경기에서 정보 추출
for cell in game_cells:
    # 경기 결과가 포함된 <a> 태그가 있는 <td>만 처리
    if cell.find_elements(By.TAG_NAME, "a"):
        # 경기 결과 정보 추출
        game_link = cell.find_element(By.TAG_NAME, "a")
        
        # 팀 이름 추출
        teams = game_link.find_elements(By.TAG_NAME, "span")[0:2]
        home_team = teams[0].text.strip()  # 예: LG
        away_team = teams[1].text.strip()  # 예: 키움

        # 점수 추출 (숫자)
        scores = game_link.find_elements(By.TAG_NAME, "em")
        home_score = scores[0].text.strip()  # 예: 2
        away_score = scores[1].text.strip()  # 예: 4

        # 경기 결과 (승패 정보) 추출
        result_img = game_link.find_element(By.CSS_SELECTOR, "img.result")
        result = result_img.get_attribute("alt")  # 예: "승" 또는 "패"

        # 경기 시간 및 장소
        location_time = game_link.find_element(By.CLASS_NAME, "location").text.strip()  # 예: "고척 18:30"

        # 결과 출력
        print(f"경기: {home_team} {home_score} : {away_team} {away_score}")
        print(f"결과: {result}")
        print(f"장소 및 시간: {location_time}")
        print("-" * 30)

# 브라우저 종료
driver.quit()


JavascriptException: Message: javascript error: {"status":11,"value":"Element is not currently visible and may not be manipulated"}
  (Session info: chrome=130.0.6723.119)
Stacktrace:
	GetHandleVerifier [0x0094EBD3+24307]
	(No symbol) [0x008D8D74]
	(No symbol) [0x007BC323]
	(No symbol) [0x007C1DA0]
	(No symbol) [0x007C3628]
	(No symbol) [0x007C36A0]
	(No symbol) [0x008011F7]
	(No symbol) [0x007F47F1]
	(No symbol) [0x0081FE8C]
	(No symbol) [0x007F3C14]
	(No symbol) [0x00820124]
	(No symbol) [0x00839579]
	(No symbol) [0x0081FC26]
	(No symbol) [0x007F219C]
	(No symbol) [0x007F311D]
	GetHandleVerifier [0x00BF8D93+2818227]
	GetHandleVerifier [0x00C5542E+3196750]
	GetHandleVerifier [0x00C4D9D2+3165426]
	GetHandleVerifier [0x009EDA70+675216]
	(No symbol) [0x008E1B3D]
	(No symbol) [0x008DEA18]
	(No symbol) [0x008DEBB5]
	(No symbol) [0x008D1640]
	BaseThreadInitThunk [0x75CB7BA9+25]
	RtlInitializeExceptionChain [0x7756C0CB+107]
	RtlClearBits [0x7756C04F+191]


In [ ]:
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))